In [ ]:
import cv2
import numpy as np
import copy
from ctypes import *
import matplotlib.pyplot as plt

import numpy.ctypeslib as npct

In [ ]:
lib_path1 = '/media/sf_evezzali/OneDrive - Unimore/Gitlab/Barcode-Localization-Benchmark/Creusot2016/build/libLCEBDFTB.so'
lib_path2 = '/media/sf_evezzali/OneDrive - Unimore/Gitlab/Barcode-Localization-Benchmark/Gallo2011-Soros2013-Yun2017/src/Linux/tencon/build/libBarcodeLocalization.so'
lib_path3 = '/media/sf_evezzali/OneDrive - Unimore/Gitlab/Barcode-Localization-Benchmark/Zamberletti2013/build/libBarcodeLibrary.so'
lib_path4 = '/media/sf_evezzali/OneDrive - Unimore/Gitlab/Barcode-Localization-Benchmark/Tekin2012/build/libblade.so'

In [ ]:
lib1 = cdll.LoadLibrary(lib_path1)
lib2 = cdll.LoadLibrary(lib_path2)
lib3 = cdll.LoadLibrary(lib_path3)
lib4 = cdll.LoadLibrary(lib_path4)

In [ ]:
#img = cv2.imread('../../PhD Resources/Datasets/Artelab 1D barcodes/Dataset1/05102009101.jpg')
img = cv2.imread('../../PhD Resources/Datasets/Artelab 1D barcodes/Dataset1/05102009081.jpg')
#img = cv2.imread('../../PhD Resources/Datasets/Artelab 1D barcodes/Dataset1/05102009190.jpg')
#img = cv2.imread('../../PhD Resources/Datasets/barcode_bb/multi_bb/multi_bb/multi_img/20200916_220016.jpg')

# Set if the image should be resized to a target size

In [ ]:
resize_image = True
target_W = 640
target_H = 480

In [ ]:
if resize_image:
    img = cv2.resize(img, (target_W, target_H))
    
plt.imshow(img)

# Gallo 2011

### Reading 1D Barcodes with Mobile Phones Using Deformable Templates 

In [ ]:
h, w, _ = img.shape
h,w = c_short(h), c_short(w)
input_img = img.ctypes.data_as(POINTER(c_uint8))
result = (c_int*8)()

lib2.galloProcess(result, input_img, h,w, c_int(20));

In [ ]:
isClosed = True
color = (255, 0, 0)
thickness = 2
image = copy.deepcopy(img)

pts = np.array(result)[:8].reshape((4, 1, 2))
image = cv2.polylines(image, [pts], isClosed, color, thickness)
    
plt.imshow(image)

# Tekin 2012

### BLaDE: Barcode Localization and Decoding Engine 

In [ ]:
h, w,_ = img.shape
h,w = c_short(h), c_short(w)
input_img = img.ctypes.data_as(POINTER(c_uint8))

result = (c_int*(500*4))()
num_results = (c_int*1)()

lib4.locateBarcodes(result, num_results, input_img, h,w);

In [ ]:
isClosed = True
color = (255, 0, 0)
thickness = 2
image = copy.deepcopy(img)

pts = np.array(result)[:num_results[0]*4].reshape((-1, 2))
for i in range(len(pts)//2):
    image = cv2.line(image,pts[i*2],pts[i*2+1],color,thickness)
#image = cv2.polylines(image, pts, isClosed, color, thickness)
    
plt.imshow(image)

# Soros 2013

### Blur-resistant joint 1D and 2D barcode localization for smartphones

In [ ]:
h, w, _ = img.shape
h,w = c_short(h), c_short(w)
input_img = img.ctypes.data_as(POINTER(c_uint8))
result = (c_int*8)()

lib2.sorosProcess(result, input_img, h,w, c_int(20));

In [ ]:
isClosed = True
color = (255, 0, 0)
thickness = 2
image = copy.deepcopy(img)

pts = np.array(result)[:8].reshape((4, 1, 2))
image = cv2.polylines(image, [pts], isClosed, color, thickness)
    
plt.imshow(image)

# Zamberletti 2013

### Robust Angle Invariant 1D Barcode Detection 

In [ ]:
h, w, _ = img.shape
h,w = c_short(h), c_short(w)
input_img = img.ctypes.data_as(POINTER(c_uint8))

result = (c_int*(100*8))()
num_results = (c_int*1)()
angle = (c_double*1)()

lib3.initialize('./Zamberletti2013/net61x3.net'.encode('utf-8'))
lib3.locateBarcode(result, angle, num_results, input_img, h,w);

In [ ]:
isClosed = True
color = (255, 0, 0)
thickness = 2
image = copy.deepcopy(img)

pts = np.array(result)[:num_results[0]*8].reshape((-1,4,2))
for i in range(len(pts)):
    center = np.mean(pts[i], axis=0)
    M = cv2.getRotationMatrix2D(center, -angle[0], 1.0)
    pts[i] = np.matmul(pts[i], M[:,:2].T)+M[:,2]
    
image = cv2.polylines(image, pts[:,:,np.newaxis,:], isClosed, color, thickness)
    
plt.imshow(image)

# Creusot 2016

### Low-Computation Egocentric Barcode Detector for the Blind 

In [ ]:
h, w, _ = img.shape
h,w = c_short(h), c_short(w)
input_img = img.ctypes.data_as(POINTER(c_uint8))

minLineLength = c_int(20)
support_candidates_threshold = c_int(10)
delta = c_int(5)
maxLengthToLineLengthRatio = c_int(30)
minLengthToLineLengthRatio = c_int(1)
inSegmentXDistance = c_int(3)
inSegmentYDistance = c_int(3)

result = (c_int*10000)()
num_results = (c_int*1)()

lib1.locateBarcode(result, num_results, input_img, h,w, minLineLength,
                 support_candidates_threshold,
                 delta, maxLengthToLineLengthRatio, minLengthToLineLengthRatio,
                 inSegmentXDistance, inSegmentYDistance);

In [ ]:
isClosed = True
color = (255, 0, 0)
thickness = 2
image = copy.deepcopy(img)

pts = np.array(result)[:num_results[0]*8].reshape((-1,4, 1, 2))
image = cv2.polylines(image, pts, isClosed, color, thickness)
    
plt.imshow(image)

# Yun 2017

### Vision-Based 1D Barcode Localization Method for Scale and Rotation Invariant 

In [ ]:
h, w, _ = img.shape
h,w = c_short(h), c_short(w)
input_img = img.ctypes.data_as(POINTER(c_uint8))
result = (c_int*100*8)()
num_results = (c_int*1)()

lib2.yunProcess(result, num_results, input_img, h,w);

In [ ]:
isClosed = True
color = (255, 0, 0)
thickness = 2
image = copy.deepcopy(img)

pts = np.array(result)[:num_results[0]*8].reshape((-1,4, 1, 2))


image = cv2.polylines(image, pts, isClosed, color, thickness)
    
plt.imshow(image)